In [1]:
#My VS Code has issues connecting to the venv.
# I find that first connecting to one of my conda environments
# and then switching to the venv works.
# I run this cell on conda environment first then switch to venv
# and run it there and seems to be all good.
print("Hello")

Hello


In [2]:
# This cell allows hot reloading of modules.
# It is useful for development, so you don't have to restart the kernel.
# Some changes such as adding new files or changing the structure of the code
# may not be picked up and will require a kernel restart.
%load_ext autoreload
%autoreload 2

In [3]:
import os
import json
import time
import geopandas as gpd
from dotenv import load_dotenv, find_dotenv
import logging

# Set up logging include logger name
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

from kapipy.gis import GIS

#import pandas as pd
#from arcgis.features import GeoAccessor, GeoSeriesAccessor

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
api_key = os.getenv('LINZ_API_KEY')

#layers
rail_station_layer_id = "50318" #rail station 175 points
fences_layer_id = "50268" #NZ Fence Centrelines  
geodetic_marks_layer_id = "50787" # NZ Geodetic Marks 132,966 point features
nz_walking_biking_tracks_layer_id = "52100" #NZ Walking and Biking Tracks 29,187 polyline features

#tables
nz_building_age_table_id = "105617" #NZ Building Age 18 records, no changeset
suburb_locality_table_id = "113761" #NZ Suburb Locality 3190 records

output_folder = r"c:\temp\data"


## Server object  
Get a reference to the LINZ server  

In [4]:
linz = GIS(name="linz", api_key=api_key)
print(linz)

GIS: LINZ at https://data.linz.govt.nz/ (API v1.x)


## Layer item  
Run some tests on a layer item  

In [5]:
itm = linz.content.get(rail_station_layer_id)
print(itm)

2025-06-08 09:54:35,002 - httpx - INFO - HTTP Request: GET https://data.linz.govt.nz/services/api/v1.x/data/?id=50318 "HTTP/1.1 200 OK"
2025-06-08 09:54:35,578 - httpx - INFO - HTTP Request: GET https://data.linz.govt.nz/services/api/v1.x/layers/50318/ "HTTP/1.1 200 OK"


VectorItem(id=50318, url='https://data.linz.govt.nz/services/api/v1.x/layers/50318/', type_='layer', title='NZ Rail Station Points (Topo, 1:50k)', description="A point on a railway line used for the picking up and setting down of passengers or freight\n\nData Dictionary for rail_station_pnt: https://docs.topo.linz.govt.nz/data-dictionary/tdd-class-rail_station_pnt.html\n\nThis layer is a component of the Topo50 map series. The Topo50 map series provides topographic mapping for the New Zealand mainland, Chatham and New Zealand's offshore islands, at 1:50,000 scale.\n\nFurther information on Topo50: \nhttp://www.linz.govt.nz/topography/topo-maps/topo50", data=VectorItemData(storage='classic', datasources='https://data.linz.govt.nz/services/api/v1.x/layers/50318/versions/412263/datasources/', fields=[FieldDef(name='GEOMETRY', type_='geometry'), FieldDef(name='t50_fid', type_='bigint'), FieldDef(name='name_ascii', type_='string'), FieldDef(name='macronated', type_='string'), FieldDef(name=

In [6]:
print(f'{itm.supports_changesets=}')
print(f'{itm.data.fields=}')
print(f'{itm.data.crs=}')
print(f'{itm.data.geometry_type=}')

2025-06-08 09:54:37,172 - httpx - INFO - HTTP Request: GET https://data.linz.govt.nz/services/api/v1.x/layers/50318/services/ "HTTP/1.1 200 OK"


itm.supports_changesets=True
itm.data.fields=[FieldDef(name='GEOMETRY', type_='geometry'), FieldDef(name='t50_fid', type_='bigint'), FieldDef(name='name_ascii', type_='string'), FieldDef(name='macronated', type_='string'), FieldDef(name='name', type_='string')]
itm.data.crs=CRS(id='EPSG:2193', url='https://data.linz.govt.nz/services/api/v1.x/coordinate-systems/EPSG:2193/', name='NZGD2000 / New Zealand Transverse Mercator 2000', kind='projected', unit_horizontal='metre', unit_vertical='', url_external='https://epsg.io/2193', component_horizontal=None, component_vertical=None, srid=2193)
itm.data.geometry_type='point'


In [ ]:
itm.data.fields[0]

In [ ]:
waikato_polygon = gpd.read_file('../examples/waikato.json')
print(waikato_polygon)

# read shapefile into a GeoDataFrame
matamata_gdf = gpd.read_file("../examples/matamata_piako.shp")
matamata_gdf.head()

In [ ]:
ext_sdf = pd.DataFrame.spatial.from_featureclass( "C:\\projects\\playground\\playground.gdb\\extent" )
ext_sdf.info()

In [ ]:
#Query using WFS
res = itm.query(count=100, wkid = 2193)
logging.info(f"Total records returned {itm.title}: {res.shape[0]}")
res.head()

In [ ]:
res.spatial.full_extent

In [ ]:
res.spatial.to_featureclass(os.path.join(output_folder, 'sdf.shp'))

In [8]:
# Get a changeset from the layer using WFS
cs_data = itm.changeset(from_time="2024-01-01T00:00:00Z", wkid=2193 )
logging.info(f"Total records returned {itm.title}: {cs_data.shape[0]}")
cs_data.head()

2025-06-08 09:55:03,266 - kapipy.conversion - INFO - Getting default output format.
2025-06-08 09:55:03,266 - kapipy.conversion - INFO - has_arcgis=False
2025-06-08 09:55:03,267 - kapipy.conversion - INFO - has_geopandas=True
2025-06-08 09:55:03,578 - root - INFO - Total records returned NZ Rail Station Points (Topo, 1:50k): 41


,__change__,t50_fid,name_ascii,macronated,name,geometry
0,DELETE,7874343,Swanson Station,N,Swanson Station,POINT (1740542.406 5918801.891)
1,DELETE,7874348,Kingsland,N,Kingsland,POINT (1755495.158 5917850.878)
2,DELETE,2462509,Sawyers Bay Station,N,Sawyers Bay Station,POINT (1413934.937 4922988.347)
3,DELETE,6455509,Sheffield,N,Sheffield,POINT (1520561.716 5195578.8)
4,DELETE,2462550,Seddon Station,N,Seddon Station,POINT (1689445.201 5385956.321)


In [ ]:
cs_data.spatial.to_featureclass(os.path.join(output_folder, 'sdf_changeset.shp'))

In [ ]:
ext_sdf.spatial.geometry_type

In [ ]:
job = itm.export("geodatabase", wkid=2193, extent=ext_sdf)

In [ ]:
print(job.status)

In [ ]:
dl = job.download(folder=output_folder)

In [ ]:
print(f'{dl.folder=}')
print(f'{dl.filename=}')
print(f'{dl.file_path=}')
print(f'{dl.file_size_bytes=}')
print(f'{dl.download_url=}')
print(f'{dl.final_url=}')
print(f'{dl.job_id=}')
print(f'{dl.completed_at=}')
print(f'{dl.checksum=}')

## Table item  
Run some tests on a table item

In [ ]:
table_itm = linz.content.get(suburb_locality_table_id)
print(table_itm)

In [ ]:
print(f'{table_itm.supports_changesets=}')
print(f'{table_itm.data.fields=}')

In [ ]:
#Query using WFS to get DataFrame
data = table_itm.query(count=100)
logging.info(f"Total records returned {table_itm.title}: {data.shape[0]}")
data.head()

In [ ]:
#Query using WFS to get raw JSON
data = table_itm.query_json(count=10)
logging.info(f"Total records returned {table_itm.title}: {len(data['features'])}")
print(data)

In [ ]:
# Get a changeset from the layer using WFS
cs_data = table_itm.get_changeset(from_time="2024-01-01T00:00:00Z", count=100)
logging.info(f"Total records returned {table_itm.title}: {cs_data.shape[0]}")
cs_data.head()

In [ ]:
# Get a changeset from the layer using WFS
cs_data = table_itm.get_changeset_json(from_time="2024-01-01T00:00:00Z", count=10)
logging.info(f"Total records returned {table_itm.title}: {len(cs_data['features'])}")
print(cs_data)

In [ ]:
job = table_itm.export("geodatabase")

In [ ]:
print(job)

In [ ]:
job.download(folder=output_folder)

## Test multidownload samples  
Test downloading multiple jobs using the ContentManager download method.    

In [ ]:
itm1 = linz.content.get(geodetic_marks_layer_id)
itm2 = linz.content.get(rail_station_layer_id)
print(itm1)
print(itm2)

In [ ]:
job1 = itm1.export("geodatabase", crs="EPSG:2193", extent=matamata_gdf,)
job2 = itm2.export("geodatabase", crs="EPSG:2193", extent=matamata_gdf,)

output_folder = r"c:\temp\data\multiple"
results = linz.content.download(jobs=[job1, job2], folder=output_folder)

In [ ]:
for result in results:
    print(result.download_file_path)
